# Google Colab Audio Recorder

This notebook demonstrates browser-based audio recording in Google Colab using JavaScript and the Web Audio API. This approach bypasses PyAudio limitations and works directly in the browser.

**🎤 Browser-Based Recording Features:**
- Direct microphone access through browser (no PyAudio needed)
- Same UI design and functionality as the original recorder
- Automatic file saving with timestamps and defect classification
- Real-time recording timer and status updates
- Compatible with Google Drive for data persistence

**📋 Instructions:**
1. **Allow Microphone Access**: Click "Allow" when your browser asks for microphone permission
2. **Select Defect Type**: Choose the appropriate classification for your recording
3. **Set Duration**: Choose recording length (1-10 seconds)
4. **Record**: Click "START RECORDING" and speak into your microphone
5. **Auto-Save**: Files are automatically saved with timestamps

In [ ]:
import os

# Clone the repository if it doesn't exist
if not os.path.exists('FrED-Predictive-Maintenance'):
    !git clone https://github.com/Ibrahimhk98/FrED-Predictive-Maintenance.git
    print("✅ Repository cloned successfully!")
else:
    print("✅ Repository already exists")

Cloning into 'FrED-Predictive-Maintenance-'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
^Cceiving objects:  22% (49/219), 102.32 MiB | 13.48 MiB/s


In [ ]:
# Install required packages (no PyAudio needed for browser recording!)
print("📦 Installing Python dependencies...")
!pip install ipywidgets numpy

print("✅ Dependencies installed - ready for browser-based recording!")

Error installing requirements: Command '['/Users/ibrahimelkhatib/miniforge3/bin/python', '-m', 'pip', 'install', '-r', '/requirements.txt']' returned non-zero exit status 1.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/requirements.txt'


In [ ]:
# Mount Google Drive for data storage
from google.colab import drive
import os

print("📁 Mounting Google Drive...")
drive.mount('/content/drive')

# Use the shared folder ID for common data storage
shared_folder_id = "1p7JVh_gL663af7pwGHQPBWmp1TiS3GPz"
data_folder = f"/content/drive/MyDrive/{shared_folder_id}"

# Create the shared folder if it doesn't exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder, exist_ok=True)
    print(f"📁 Created shared audio data folder: {data_folder}")
else:
    print(f"✅ Using shared data folder: {data_folder}")

print(f"🔗 Shared Folder ID: {shared_folder_id}")
print("🎵 Audio files will be organized by defect type automatically!")
print("👥 All students will save to the same shared location for easy access")

In [ ]:
# Verify shared folder setup and show structure
print("🔍 Verifying shared folder setup...")
print(f"📍 Shared Folder Path: {data_folder}")

# Check if the folder is accessible
if os.path.exists(data_folder):
    print("✅ Shared folder is accessible")
    
    # Show existing structure if any
    try:
        contents = os.listdir(data_folder)
        if contents:
            print(f"📂 Existing contents ({len(contents)} items):")
            for item in sorted(contents):
                item_path = os.path.join(data_folder, item)
                if os.path.isdir(item_path):
                    file_count = len([f for f in os.listdir(item_path) if f.endswith('.wav')])
                    print(f"   📁 {item}/ ({file_count} audio files)")
                else:
                    print(f"   📄 {item}")
        else:
            print("📂 Shared folder is empty - ready for new recordings")
    except Exception as e:
        print(f"⚠️ Could not list contents: {e}")
else:
    print("❌ Shared folder not accessible - creating it now...")
    os.makedirs(data_folder, exist_ok=True)

print("\n" + "="*60)
print("🎯 SHARED FOLDER CONFIGURATION")
print("="*60)
print(f"🆔 Folder ID: {shared_folder_id}")
print(f"📍 Full Path: {data_folder}")
print("👥 All student recordings will be saved here")
print("🎵 Files organized by defect type automatically")
print("="*60)

In [ ]:
# Add the repository to Python path and import the Colab audio recorder
import sys
sys.path.append('/content/FrED-Predictive-Maintenance')

# Import the Colab-compatible audio recorder
from Preparation.Import.colab_audio_recorder_fixed import create_recorder_ui, list_colab_audio_devices

# Show available audio capabilities
print("🎤 Audio Device Information:")
print("=" * 50)
for info in list_colab_audio_devices():
    print(info)

print("\n" + "="*60)
print("🚀 Launching Browser-Based Audio Recorder...")
print("="*60)
print("✅ ENHANCED FEATURES:")
print("   • Files save to Drive folder properly")
print("   • Duration extended to 400 seconds")  
print("   • Team numbers included in filenames")
print("   • Non-blocking JavaScript execution")
print("   • Audio device info displayed")
print("="*60)

# Create and display the recorder UI with team number support
# You can specify team number here: team_number=1, 2, 3, etc.
ui = create_recorder_ui(base_dir=data_folder, team_number=None)  # Set team_number as needed
display(ui)

0: Ibrahim’s iPhone Microphone
2: MacBook Pro Microphone
4: Immersed


## 🔧 Recording Issues? Try the Working Version!

If the audio recorder above isn't working properly, try this enhanced version with comprehensive debugging and multiple fallback mechanisms.

In [ ]:
# Import the WORKING version with comprehensive debugging
from Preparation.Import.colab_audio_recorder_working import create_recorder_ui as create_working_recorder

print("🔧 TROUBLESHOOTING AUDIO RECORDER")
print("=" * 50)
print("🎯 This version includes:")
print("   • Comprehensive debugging output")
print("   • Multiple fallback mechanisms") 
print("   • Real-time error reporting")
print("   • System test functionality")
print("   • Enhanced microphone handling")
print("=" * 50)

# Create the working recorder with debugging enabled
print("\n🚀 Loading Working Audio Recorder...")
working_ui = create_working_recorder(base_dir=data_folder, team_number=None)
display(working_ui)

print("\n📋 TROUBLESHOOTING STEPS:")
print("1. Click 'TEST SYSTEM' first to verify all components")
print("2. Check debug output for any error messages")
print("3. Allow microphone access when prompted")
print("4. Try a short recording (3-5 seconds) first")
print("5. Check the debug output for detailed status messages")

In [ ]:
# DIAGNOSTIC CELL - Run this to identify recording issues
print("🔍 AUDIO RECORDING DIAGNOSTICS")
print("=" * 50)

# Check 1: Google Colab environment
try:
    from google.colab import output, drive
    print("✅ Google Colab: Available")
    colab_status = "OK"
except ImportError:
    print("❌ Google Colab: Not available (running locally?)")
    colab_status = "FAIL"

# Check 2: Drive mounting
if os.path.exists('/content/drive'):
    print("✅ Google Drive: Mounted")
    drive_status = "OK"
else:
    print("❌ Google Drive: Not mounted")
    drive_status = "FAIL"

# Check 3: Data folder accessibility
if os.path.exists(data_folder):
    print(f"✅ Data folder: Accessible ({data_folder})")
    # Check write permissions
    try:
        test_file = os.path.join(data_folder, "test_write.tmp")
        with open(test_file, 'w') as f:
            f.write("test")
        os.remove(test_file)
        print("✅ Write permissions: OK")
        folder_status = "OK"
    except Exception as e:
        print(f"❌ Write permissions: FAILED - {e}")
        folder_status = "FAIL"
else:
    print(f"❌ Data folder: Not accessible ({data_folder})")
    folder_status = "FAIL"

# Check 4: JavaScript capabilities
from IPython.display import Javascript
display(Javascript("""
    console.log('🧪 JavaScript Diagnostic Test');
    
    // Check MediaRecorder support
    if (typeof MediaRecorder !== 'undefined') {
        console.log('✅ MediaRecorder: Supported');
    } else {
        console.log('❌ MediaRecorder: Not supported');
    }
    
    // Check getUserMedia support
    if (navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
        console.log('✅ getUserMedia: Supported');
    } else {
        console.log('❌ getUserMedia: Not supported');
    }
    
    // Check Google Colab API
    if (typeof google !== 'undefined' && google.colab) {
        console.log('✅ Google Colab API: Available');
    } else {
        console.log('❌ Google Colab API: Not available');
    }
    
    console.log('📋 Check the browser console (F12) for these diagnostic messages');
"""))

print("\n📊 DIAGNOSTIC SUMMARY:")
print(f"   Colab Environment: {colab_status}")
print(f"   Google Drive: {drive_status}")
print(f"   Data Folder: {folder_status}")
print("   JavaScript: Check browser console (F12)")

print("\n🔧 NEXT STEPS:")
if colab_status == "FAIL":
    print("   • You must run this in Google Colab")
if drive_status == "FAIL":
    print("   • Run the Google Drive mounting cell first")
if folder_status == "FAIL":
    print("   • Check the data_folder path and permissions")

print("\n💡 If all checks pass but recording still fails:")
print("   • Try the working recorder above with debug output")
print("   • Check browser microphone permissions")
print("   • Use the TEST SYSTEM button in the working recorder")